In [7]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        '''
        self.upper_layer1 = nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.upper_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.under_layer1 = nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.ReLU()
        )
        self.final_layer = nn.Conv2d(224,5,21)
        '''
        self.upper_conv1 = nn.Conv2d(4,64,7)
        self.upper_conv2 = nn.Conv2d(64,64,3)
        self.under_conv1 = nn.Conv2d(4,160,13)
        self.final_layer = nn.Conv2d(224,5,21)
        
    def forward(self, x):
        #upper_x = self.upper_layer2(self.upper_layer1(x))
        #under_x = self.under_layer1(x)
        upper_x = F.max_pool2d(F.relu(self.upper_conv1(x)), (4, 4),stride = 1)
        upper_x = F.max_pool2d(F.relu(self.upper_conv2(upper_x)), (2, 2), stride = 1)
        under_x = F.relu(self.under_conv1(x))
        final_x = torch.cat((under_x, upper_x), 1)
        out = self.final_layer(final_x)
        #out = F.softmax(self.final_layer(final_x))
        out = out.view(-1,5)
        return out
        
net = TwoPathConv()
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)
#y_pred = y_pred.data.resize_(1,5)
#y_pred = Variable(y_pred,requires_grad = True)
#print(y_pred.size())
#print(y_pred.view)

TwoPathConv (
  (upper_conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
  (upper_conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (under_conv1): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)


In [2]:
f=open("trainval-balanced.txt", "r")
content=f.readlines()
data=[]
data_train=[]
data_val=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    data.append([])
    data[i].append(item[0])
    data[i].append(int(item[1]))
    data[i].append(int(item[2]))
    data[i].append(int(item[3]))
    data[i].append(int(item[4]))
    if i%10000==0:
        data_val.append(data[i])
    else:
        data_train.append(data[i])
    i += 1
f.close()
import h5py
f = h5py.File('training.h5','r')

In [17]:
print len(data)
print len(data_val)
print len(data_train)

8998296
900
8997396


In [3]:
print torch.cuda.is_available()

True


In [4]:
import torch.nn.init as ninit
def init_net(net):
    for param in net.parameters():
        ninit.uniform(param.data, a=-5e-3, b=5e-3)

In [5]:
def create_batch(batch_mask):
    X_batch = []
    y_batch = []
    for i in range(len(batch_mask)):
        case, x, y, z, l = data_train[batch_mask[i]]
        case1 = case[:2]
        case2 = case[3:]
        X_batch.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_batch.append(l)
    X_batch = torch.from_numpy(np.array(X_batch))
    y_batch = torch.from_numpy(np.array(y_batch))
    return X_batch, y_batch

def create_val(batch_mask):
    X_val=[]
    y_val=[]
    for i in range(len(batch_mask)):
        case, x, y, z, l = data_val[batch_mask[i]]
        case1 = case[:2]
        case2 = case[3:]
        X_val.append(f[case1][case2][:, x-16:x+17, y-16:y+17, z])
        y_val.append(l)
    X_val = torch.from_numpy(np.array(X_val))
    y_val = torch.from_numpy(np.array(y_val))
    return X_val, y_val

In [12]:
import time
import numpy as np
num_train=len(data_train)
num_val=len(data_val)
batch_size=516
val_size=num_val
num_epoch=100.0
num_times=int(float(num_train)/batch_size*num_epoch)
print num_times
learning_rate = 5e-3
reg=1e-4
net = TwoPathConv()
print(net)
init_net(net)
net.cuda(2)
prev_time = time.clock()
for i in xrange(1001):  # loop over the dataset multiple times
    batch_mask = np.random.choice(num_train, batch_size)
    X_batch=None
    X_batch, y_batch = create_batch(batch_mask)
    X_batch, y_batch = Variable(X_batch.cuda(2)), Variable(y_batch.cuda(2), requires_grad = False)
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=reg)
    optimizer.zero_grad()
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()
    if i % 10 == 0:
        print loss
        print 'time used %.3f' % (time.clock()-prev_time)
        val_mask=np.random.choice(num_val, val_size)
        X_val, y_val=create_val(val_mask)
        X_val=Variable(X_val.cuda(2), requires_grad=False)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(2)).sum()
        print('Validation accuracy:', float(correct)/val_size)
torch.save(net.state_dict(), "premature_net.txt")
print "success"

1743681
TwoPathConv (
  (upper_conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
  (upper_conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (under_conv1): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)
Variable containing:
 1.6118
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 1.462
('Validation accuracy:', 0.25666666666666665)
Variable containing:
 1.4074
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 15.231
('Validation accuracy:', 0.4177777777777778)
Variable containing:
 1.2582
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 29.204
('Validation accuracy:', 0.46444444444444444)
Variable containing:
 1.1934
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 43.202
('Validation accuracy:', 0.5011111111111111)
Variable containing:
 1.0887
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 56.610
('Validation accuracy:', 0.5788888888888889)
Variable containing:
 1.0599


In [30]:
def adam(x, dx, config = None):
    if config is None: 
        config = {}
        config.setdefault('learning_rate',1e-4)
        config.setdefault('beta1',0.9)
        config.setdefault('beta2',0.999)
        config.setdefault('epsilon',1e-8)
        config.setdefault('m',torch.FloatTensor(x.size()).zero_().cuda(2))
        config.setdefault('v',torch.FloatTensor(x.size()).zero_().cuda(2))
        config.setdefault('t',0)
    
    next_x = None
    config['t'] += 1
    x.cuda(2)
    dx.cuda(2)
    config['m'] = config['beta1'] * config['m'] + (1 - config['beta1']) * dx
    config['v'] = config['beta2'] * config['v'] + (1 - config['beta2']) * dx**2
    mb = config['m'] / (1 - config['beta1'] ** config['t'])
    vb = config['v'] / (1 - config['beta2'] ** config['t'])
    next_x = x - config['learning_rate'] * mb / (np.sqrt(vb) + config['epsilon'])
    return next_x, config

In [8]:
f_in=open("trainval.txt", "r")
content=f_in.readlines()
data=[]
data_train=[]
data_val=[]
i=0
for line in content:
    no_n_line=line[0:len(line)-1]
    item=no_n_line.split(" ")
    data.append([])
    data[i].append(item[0])
    data[i].append(int(item[1]))
    data[i].append(int(item[2]))
    data[i].append(int(item[3]))
    data[i].append(int(item[4]))
    if i%30000==0:
        data_val.append(data[i])
    else:
        data_train.append(data[i])
    i += 1
f_in.close()

In [9]:
print len(data)
print len(data_val)
print len(data_train)

24778697
826
24777871


In [10]:
num_train=len(data_train)
num_val=len(data_val)
batch_size=900
val_size=num_val
num_epoch=10.0
num_times=int(float(num_train)/batch_size*num_epoch)
print num_times
learning_rate = 5e-3
net.load_state_dict(torch.load("premature_net.txt"))
prev_time = time.clock()
config_weight = None
config_bias = None
for i in xrange(101):  # loop over the dataset multiple times
    batch_mask = np.random.choice(num_train, batch_size)
    X_batch = None
    X_batch, y_batch = create_batch(batch_mask)
    X_batch, y_batch = Variable(X_batch.cuda(2)), Variable(y_batch.cuda(2), requires_grad = False)
    y_pred = net.forward(X_batch)
    y_pred = y_pred.view(-1,5)
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    dweight = net.final_layer.weight.grad.data
    dbias = net.final_layer.bias.grad.data
    net.final_layer.weight.data, config_weight = adam(net.final_layer.weight.data, dweight, config_weight)
    net.final_layer.bias.data, config_bias = adam(net.final_layer.bias.data, dbias, config_bias)
    if i % 10 == 0:
        print loss
        print 'time used %.3f' % (time.clock()-prev_time)
        val_mask=np.random.choice(num_val, val_size)
        X_val, y_val=create_val(val_mask)
        X_val=Variable(X_val.cuda(2), requires_grad=False)
        y_val_pred=net.forward(X_val)
        y_val_pred=y_val_pred.view(-1,5)
        useless, predicted=torch.max(y_val_pred.data, 1)
        correct = (predicted == y_val.cuda(2)).sum()
        print('Validation accuracy:', float(correct)/val_size)
print "successfully trained"

275309
Variable containing:
 1.1957
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 6.262
('Validation accuracy:', 0.7288135593220338)
Variable containing:
 1.1337
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 33.144
('Validation accuracy:', 0.8147699757869249)
Variable containing:
 1.0669
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 56.063
('Validation accuracy:', 0.8462469733656174)
Variable containing:
 1.0381
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 78.881
('Validation accuracy:', 0.8801452784503632)
Variable containing:
 1.0326
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 101.827
('Validation accuracy:', 0.8559322033898306)
Variable containing:
 1.0259
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 124.738
('Validation accuracy:', 0.8813559322033898)
Variable containing:
 1.0382
[torch.cuda.FloatTensor of size 1 (GPU 2)]

time used 147.661
('Validation accuracy:', 0.8692493946731235)
Variable containing:
 1.0315
[torch.cuda.Float